In [1]:
from marubatsu import Marubatsu
from ai import ai_by_score

def ai5s(mb, debug=False):
    def eval_func(mb):
        my_turn = Marubatsu.CROSS if mb.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE
        if mb.status == my_turn:
            return 1
        else:
            return 0

    return ai_by_score(mb, eval_func, debug=debug)

In [2]:
from ai import ai_match, ai5, ai6, ai7

ai_match(ai=[ai5s, ai5])

ai5s VS ai5
count     win    lose    draw
o        6754    2835     411
x        2646    6916     438
total    9400    9751     849

ratio     win    lose    draw
o       67.5%   28.3%    4.1%
x       26.5%   69.2%    4.4%
total   47.0%   48.8%    4.2%



In [3]:
def ai5s(mb, debug=False):
    def eval_func(mb):
        if mb.status == Marubatsu.CIRCLE or mb.status == Marubatsu.CROSS:
            return 1
        else:
            return 0

    return ai_by_score(mb, eval_func, debug=debug)

In [4]:
ai_match(ai=[ai5s, ai5])

ai5s VS ai5
count     win    lose    draw
o        6881    2661     458
x        2756    6831     413
total    9637    9492     871

ratio     win    lose    draw
o       68.8%   26.6%    4.6%
x       27.6%   68.3%    4.1%
total   48.2%   47.5%    4.4%



In [5]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1 
    self.last_turn = None

Marubatsu.restart = restart

In [6]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.status = self.judge()
        self.last_move = x, y

Marubatsu.move = move

In [7]:
def ai5s(mb, debug=False):
    def eval_func(mb):
        if mb.status == mb.last_turn:
            return 1
        else:
            return 0

    return ai_by_score(mb, eval_func, debug=debug)

In [8]:
ai_match(ai=[ai5s, ai5])

ai5s VS ai5
count     win    lose    draw
o        6924    2624     452
x        2761    6826     413
total    9685    9450     865

ratio     win    lose    draw
o       69.2%   26.2%    4.5%
x       27.6%   68.3%    4.1%
total   48.4%   47.2%    4.3%



In [9]:
ai_match(ai=[ai5, ai5])

ai5 VS ai5
count     win    lose    draw
o        6762    2799     439
x        2839    6744     417
total    9601    9543     856

ratio     win    lose    draw
o       67.6%   28.0%    4.4%
x       28.4%   67.4%    4.2%
total   48.0%   47.7%    4.3%



In [10]:
ai_match(ai=[ai5s, ai5s])

ai5s VS ai5s
count     win    lose    draw
o        6798    2755     447
x        2641    6920     439
total    9439    9675     886

ratio     win    lose    draw
o       68.0%   27.6%    4.5%
x       26.4%   69.2%    4.4%
total   47.2%   48.4%    4.4%



In [11]:
mb = Marubatsu()

mb.move(1, 1)
mb.move(1, 0)
mb.move(0, 0)
mb.move(2, 0)
mb.move(0, 1)
mb.move(1, 2)
print(mb)

print("ai5")
for i in range(10):
    print(ai5(mb))
    
print("ai5s")
for i in range(10):
    print(ai5s(mb))

Turn o
oxx
oo.
.X.

ai5
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
ai5s
(2, 2)
(2, 1)
(2, 1)
(0, 2)
(2, 2)
(2, 1)
(2, 2)
(2, 2)
(2, 1)
(2, 2)


In [12]:
from collections import defaultdict

def count_marks(self, coord, dx, dy):
    x, y = coord   
    count = defaultdict(int)
    for _ in range(self.BOARD_SIZE):
        count[self.board[x][y]] += 1
        x += dx
        y += dy

    return count

Marubatsu.count_marks = count_marks

In [13]:
mb = Marubatsu()

print(mb)
print(mb.count_marks((0, 0), 1, 0))

mb.move(0, 0)
print(mb)
print(mb.count_marks((0, 0), 1, 0))

mb.move(1, 0)
print(mb)
print(mb.count_marks((0, 0), 1, 0))

mb.move(2, 0)
print(mb)
print(mb.count_marks((0, 0), 1, 0))

Turn o
...
...
...

defaultdict(<class 'int'>, {'.': 3})
Turn x
O..
...
...

defaultdict(<class 'int'>, {'o': 1, '.': 2})
Turn o
oX.
...
...

defaultdict(<class 'int'>, {'o': 1, 'x': 1, '.': 1})
Turn x
oxO
...
...

defaultdict(<class 'int'>, {'o': 2, 'x': 1})


In [14]:
def ai6s(mb, debug=False):
    def eval_func(mb):
        # 自分が勝利している場合は、評価値として 1 を返す
        if mb.status == mb.last_turn:
            return 1

        # 相手の手番で相手が勝利できる場合は評価値として -1 を返す
        # 横方向と縦方向の判定
        for i in range(mb.BOARD_SIZE):
            count = mb.count_marks(coord=[0, i], dx=1, dy=0)
            if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
                return -1
            count = mb.count_marks(coord=[i, 0], dx=0, dy=1)
            if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
                return -1
        # 左上から右下方向の判定
        count = mb.count_marks(coord=[0, 0], dx=1, dy=1)
        if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
            return -1
        # 右上から左下方向の判定
        count = mb.count_marks(coord=[2, 0], dx=-1, dy=1)
        if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
            return -1

        # それ以外の場合は評価値として 0 を返す
        return 0

    return ai_by_score(mb, eval_func, debug=debug)        

In [15]:
ai_match(ai=[ai6s, ai6])

ai6s VS ai6
count     win    lose    draw
o        3090    1750    5160
x        1818    3085    5097
total    4908    4835   10257

ratio     win    lose    draw
o       30.9%   17.5%   51.6%
x       18.2%   30.9%   51.0%
total   24.5%   24.2%   51.3%



In [16]:
mb = Marubatsu()

mb.move(1, 1)
mb.move(1, 0)
mb.move(0, 0)
mb.move(2, 0)
mb.move(0, 1)
print(mb)

print("ai6")
for i in range(10):
    print(ai6(mb))
    
print("ai6s")
for i in range(10):
    print(ai6s(mb))

Turn x
oxx
Oo.
...

ai6
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
ai6s
(1, 2)
(2, 1)
(2, 1)
(2, 1)
(2, 2)
(1, 2)
(2, 1)
(0, 2)
(1, 2)
(2, 2)


In [17]:
def ai7s(mb, debug=False):
    def eval_func(mb):
        # 真ん中のマスに着手している場合は、評価値として 2 を返す
        if mb.last_move == (1, 1):
            return 2
    
        # 自分が勝利している場合は、評価値として 1 を返す
        if mb.status == mb.last_turn:
            return 1

        # 相手の手番で相手が勝利できる場合は評価値として -1 を返す
        # 横方向と縦方向の判定
        for i in range(mb.BOARD_SIZE):
            count = mb.count_marks(coord=[0, i], dx=1, dy=0)
            if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
                return -1
            count = mb.count_marks(coord=[i, 0], dx=0, dy=1)
            if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
                return -1
        # 左上から右下方向の判定
        count = mb.count_marks(coord=[0, 0], dx=1, dy=1)
        if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
            return -1
        # 右上から左下方向の判定
        count = mb.count_marks(coord=[2, 0], dx=-1, dy=1)
        if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
            return -1

        # それ以外の場合は評価値として 0 を返す
        return 0

    return ai_by_score(mb, eval_func, debug=debug)  

In [18]:
ai_match(ai=[ai7s, ai7])

ai7s VS ai7
count     win    lose    draw
o        2995     389    6616
x         393    2981    6626
total    3388    3370   13242

ratio     win    lose    draw
o       29.9%    3.9%   66.2%
x        3.9%   29.8%   66.3%
total   16.9%   16.9%   66.2%

